<a href="https://colab.research.google.com/github/penny1xu/RESTS/blob/main/python_rests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# data clean 

BathroomsFull
BathroomsHalf
BathroomsTotalDecimal
BathroomsTotalInteger
BedroomsTotal
BuildingAreaTotal
City
CloseDate
ClosePrice
CumulativeDaysOnMarket
DaysOnMarket
GarageSpaces
Latitude
ListingContractDate
ListingId
ListPrice
ListSource
Longitude
LotSizeAcres
MlsStatus
OwnerName
OwnerPhone
OwnerPhoneAlternative
ParcelNumber
ParcelNumber2
PoolYN
PostalCode
PreviousListPrice
PreviousStatus
PropertySubType
PropertyType
PublicRemarks
PurchaseContractDate
SchoolDistrict
StandardStatus
StatusChangeTimestamp
StreetDirPrefix
StreetDirSuffix
StreetName
StreetNumber
StreetNumberNumeric
StreetSuffix
SubdivisionName
USProperty_MUI
YearBuilt



In [1]:
#import package
!pip install censusgeocode
import csv
import pandas as pd
import censusgeocode as cg
import numpy as np
import sys


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# Data cleaning

In [2]:
import pandas as pd
import censusgeocode as cg
data= pd.read_csv("final.csv")
# select the data columns we want 
data = data[['BathroomsFull','BathroomsHalf', 'BathroomsTotalDecimal', 'BathroomsTotalInteger','BedroomsTotal', 'BuildingAreaTotal', 'City', 'CloseDate', 'ClosePrice', 'CumulativeDaysOnMarket', 
   'DaysOnMarket', 'GarageSpaces', 'Latitude', 'ListingContractDate', 'ListingId', 'ListPrice', 
   'ListSource', 'LivingArea', 'Longitude', 'LotSizeAcres', 'MlsStatus', 'OwnerName', 'OwnerPhone', 'OwnerPhoneAlternative',
   'ParcelNumber', 'ParcelNumber2', 'PoolYN', 'PostalCode', 'PreviousListPrice', 'PreviousStatus', 'PropertySubType',
   'PropertyType', 'PublicRemarks', 'PurchaseContractDate', 'SchoolDistrict', 'StandardStatus', 'StatusChangeTimestamp',
   'StreetDirPrefix', 'StreetDirSuffix', 'StreetName', 'StreetNumber', 'StreetNumberNumeric', 'StreetSuffix', 'SubdivisionName',
   'USProperty_MUI', 'YearBuilt']]
code = []
long = []
lat = []
area = []
price = []
for item in data['PostalCode']:
    item = str(item)
    code.append(item)
for item in data['Longitude']:
    item = float(item)
    long.append(item)
for item in data['Latitude']:
    item = float(item)
    lat.append(item)
for item in data['LivingArea']:
    item = float(item)
    area.append(item)
for item in data['ClosePrice']:
    item = float(item)
    price.append(item)
data['PostalCode'] = code
data['Longitude'] = long
data['Latitude'] = lat
data['LivingArea'] = area
data['ClosePrice'] = price
def findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, data):
    basic = house_leftSqt
    result = data[(data['Longitude'] >= house_leftLo) & (data['Longitude'] <= house_rightLo) & (data['Latitude'] >= house_leftLa) & (data['Latitude'] <= house_rightLa) & (data['LivingArea'] >= house_leftSqt) & (data['LivingArea'] <= house_rightSqt)]
    if(len(result['Longitude']) >= 5):
        return result
    else:
        house_leftLa -= 0.01
        house_rightLa += 0.01
        house_leftLo -= 0.01
        house_rightLo += 0.01
        if(basic <= 1100):
            house_leftSqt -= 100
            house_rightSqt += 100
        elif(basic <= 2200):
            house_leftSqt -= 200
            house_rightSqt += 200
        else:
            house_leftSqt -= 300
            house_rightSqt += 300
        return findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, data)  
def findifNull(lo_left, lo_right, la_left, la_right, data):
    new_data = data[(data['Longitude'] >= lo_left) & (data['Longitude'] <= lo_right)&(data['Latitude'] >= la_left)& (data['Latitude'] <=la_right)]
    if(len(new_data['Longitude'])>=5):
        return new_data
    else:
        lo_left -= 0.01
        lo_right += 0.01
        la_left -= 0.02
        la_right += 0.02
        return findifNull(lo_left, lo_right, la_left, la_right, data)
def main(data):
    print('Enter #MLS')
    mls = input()
    #14730621
    if int(mls) in list(data['ListingId']):
        print("\nThis value exists in Dataframe")
  # get whole row value
        pulled_value = data.loc[data['ListingId'] == int(mls)]
  #get each value
        records = pulled_value.to_dict('records')
        zip = records[0].get('PostalCode')
        print(records[0])
        records = pulled_value.to_dict('records')
        data = data[data['PostalCode'] == zip ]
        house_leftLa = records[0].get('Latitude')
        house_rightLa = records[0].get('Latitude') 
        house_leftLo = records[0].get('Longitude')
        house_rightLo = records[0].get('Longitude')
        house_leftSqt = records[0].get('LivingArea')
        house_rightSqt = records[0].get('LivingArea')
        result = findHouse(house_leftLa, house_rightLa, house_leftLo, house_rightLo, house_leftSqt, house_rightSqt, data)
        mean = result['ClosePrice'].mean()
        top = list(result['ClosePrice'])
        top.sort(reverse = True)
        return result.to_csv('result.csv'), mean, top[0:3]
    else:
        print('invalid ID, please insert specific value')
        print('Enter address')
        address = input()
        #'2836 Meadowbrook Drive'
        print('Enter city')
        city = input()
        #'Dallas'
        print('Enter state')
        state = input()
        #'Texas'
        print('Enter zipcode')
        zip = input()
        #'75227'
        result = cg.address(address, city = city, state= state,zipcode = zip)
        lo_left = result[0]['coordinates']['x']
        lo_right = result[0]['coordinates']['x']
        la_left = result[0]['coordinates']['y']
        la_right = result[0]['coordinates']['y']
        data = data[data['PostalCode'] == zip ]
        result = findifNull(lo_left, lo_right, la_left, la_right, data)
        mean = result['ClosePrice'].mean()
        top = list(result['ClosePrice'])
        top.sort(reverse = True)
        return result.to_csv('result.csv'), mean, top[0:3]
  # # limit zipcode range
    
if __name__ == "__main__":
    print(main(data))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (23,24,25,27,29,30,31,76,92,132,137,139,171,173,197,202,226,227,228,229,253,255,299,300,355,359,360,361,362,379,382,383) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Enter #MLS
20070854

This value exists in Dataframe
{'BathroomsFull': 4, 'BathroomsHalf': 1, 'BathroomsTotalDecimal': 4.1, 'BathroomsTotalInteger': 5, 'BedroomsTotal': 4, 'BuildingAreaTotal': nan, 'City': 'Southlake', 'CloseDate': nan, 'ClosePrice': nan, 'CumulativeDaysOnMarket': 0, 'DaysOnMarket': 0, 'GarageSpaces': 2.0, 'Latitude': 32.942335, 'ListingContractDate': '2022-05-31', 'ListingId': 20070854, 'ListPrice': 2500000.0, 'ListSource': nan, 'LivingArea': 4150.0, 'Longitude': -97.126316, 'LotSizeAcres': 0.076, 'MlsStatus': 'Active', 'OwnerName': 'Mike & Ditto Tamer', 'OwnerPhone': nan, 'OwnerPhoneAlternative': nan, 'ParcelNumber': '41281969', 'ParcelNumber2': nan, 'PoolYN': 1.0, 'PostalCode': '76092', 'PreviousListPrice': nan, 'PreviousStatus': nan, 'PropertySubType': 'Single Family Residence', 'PropertyType': 'Residential', 'PublicRemarks': 'Absolutely stunning one-of-a-kind FULLY FURNISHED modern home completely remodeled in 2021 to include a new roof. You will never see another 